In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict, extract_features_2
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Normal CNN 10-dim vector

In [2]:
# load clean gdp data, keep only year, region and real_gdp columns
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
log_bin_columns = ["allangle_snow_free" + "_log_" + str(i) for i in range(1, 11)]

# delete Kyiv and Kyiv_Oblast
# ukraine = ukraine[ukraine["region"] != "Kyiv"]
# ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast"]
ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]

# get the data for 2021, 2022 and before 2022
ukraine_2022 = ukraine[ukraine["year"] == 2022]
ukraine = ukraine[ukraine["year"] < 2022]
ukraine_2021 = ukraine[ukraine["year"] == 2021]
ukraine_2021.reset_index(drop=True, inplace=True)
ukraine_2022.reset_index(drop=True, inplace=True)
ukraine.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(ukraine), 765, 1076, 1))
y = np.zeros(len(ukraine))

# load the images
for i in range(len(ukraine)):

    # get year, region, and gdp
    year = ukraine["year"][i]
    region = ukraine["region"][i]
    gdp_value = ukraine["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations in the test and train sets
test_year = int(2017)
test_indices = np.where(ukraine["year"] == test_year)[0]
train_indices = np.where(ukraine["year"] != test_year)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(ukraine_2022), 765, 1076, 1))
for i in range(len(ukraine_2022)):

    year = ukraine_2022["year"][i]
    region = ukraine_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [3]:
# define the grid of parameters: network architecture and number of features
param_grid_cnn = {
    'n_features': [4],
    'n_conv': [2],
    'n_dense': [4],
}

# df to store the results
cnn_results = pd.DataFrame(columns = ["n_features", "n_conv", "n_dense", "mae", "mpe", "gdp_change"])

# train first stage, neural network
for n_features in param_grid_cnn['n_features']:
    for n_conv in param_grid_cnn['n_conv']:
        for n_dense in param_grid_cnn['n_dense']:
            set_seed(0)
            model = define_cnn(n_features = n_features, n_conv = n_conv, n_dense = n_dense)
            model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
            model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

            # extract features
            # ukraine_stage_2, ukraine_2021_fea, ukraine_2022_fea = extract_features(model, ukraine, ukraine_2021, ukraine_2022, X_train, X_pred, X_test, n_features)
            ukraine_stage_2, ukraine_test = extract_features_2(model, ukraine, test_year, X_train, X_test, n_features)
            # del model

            # train second stage, rf
            train_data = ukraine_stage_2
            test_data = ukraine_test
            # test_data = ukraine_2021_fea
            # prediction_data = ukraine_2022_fea
            # pre_war_data = pd.concat([train_data, test_data])
            selected_columns =  ["feature_" + str(i) for i in range(1, n_features+1)] + ["allangle_snow_free_hq_sum", "allangle_snow_free_hq_mean"] + log_bin_columns

            # param_grid_xgb = {
            #     'eta': [0.01, 0.1, 0.2, 0.3],
            #     'gamma': [10, 20, 50, 100],
            #     'max_depth': [4, 6, 8, 10],
            #     'min_child_weight': [3, 4, 5, 6],
            #     'random_state': [0] 
            # }

            # param_grid_xgb = {
            #     'eta': [0.01, 0.1, 0.2, 0.3],
            #     'gamma': [10, 20, 50, 100],
            #     'max_depth': [4, 6, 8, 10],
            #     'min_child_weight': [3, 4, 5, 6],
            #     'random_state': [0] 
            # }

            param_grid_rf = {
                    'n_estimators': [50, 100, 200, 300],
                    'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
                    'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
                    'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
                    'random_state': [0],  # Ensures reproducibility
            }

            # mae, mpe, _, pred = build_model(train_data, test_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)
            # gdp_change, best_params = predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)

            # mae, mpe, gdp_change, gdp_predictions = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "xgboost", param_grid_xgb, log_transform = False, scale = False, total_metrics = True)
            # calculate differences
            ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
            ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]
            ukraine_sum = ukraine[['year', 'region', 'allangle_snow_free_hq_sum', 'allangle_snow_free_hq_mean'] + log_bin_columns]
            ukraine = ukraine[ukraine["year"] < 2022]

            full_data = pd.concat([train_data, test_data])
            full_data.sort_values(by=['region', 'year'], inplace=True)
            full_data = pd.merge(full_data, ukraine_sum, on=['year', 'region'])
            full_data_diff = full_data.groupby('region').diff()
            full_data_diff['region'] = full_data['region']
            full_data_diff['year'] = full_data['year']
            full_data_diff.reset_index(drop=True, inplace=True)
            full_data_diff.dropna(inplace=True)

            train_data_diff = full_data_diff[full_data_diff["year"] != test_year]
            test_data_diff = full_data_diff[full_data_diff["year"] == test_year]
            test_data_diff.reset_index(drop=True, inplace=True)
            train_data_diff.reset_index(drop=True, inplace=True)

            # pre_war_data.sort_values(by=['region', 'year'], inplace=True)
            # pre_war_data = pd.merge(pre_war_data, ukraine_sum, on=['year', 'region'])
            # pre_war_data_diff = pre_war_data.groupby('region').diff()
            # pre_war_data_diff['region'] = pre_war_data['region']
            # pre_war_data_diff['year'] = pre_war_data['year']
            # pre_war_data_diff.reset_index(drop=True, inplace=True)
            # pre_war_data_diff.dropna(inplace=True)
            
            # prediction_data = pd.concat([ukraine_2021_fea, ukraine_2022_fea])
            # prediction_data.sort_values(by=['region', 'year'], inplace=True)
            # prediction_data['real_gdp'] = 0
            # prediction_data = pd.merge(prediction_data, ukraine_sum, on=['year', 'region'])
            # prediction_data_diff = prediction_data.groupby('region').diff()
            # prediction_data_diff['region'] = prediction_data['region']
            # prediction_data_diff['year'] = prediction_data['year']
            # prediction_data_diff.reset_index(drop=True, inplace=True)
            # prediction_data_diff.dropna(inplace=True)

            # mae, mpe, gdp_change, gdp_predictions, best_params, metrics = build_model_and_predict(pre_war_data_diff, prediction_data_diff, selected_columns, "random_forest", param_grid_rf, 
            #                                                                                       log_transform = False, scale = False, total_metrics = True, diff = True)
            # gdp_per_change = sum(gdp_predictions)/ukraine_2021["real_gdp"].sum()
            mae, _, _ = build_model(train_data_diff, test_data_diff, selected_columns, "random_forest", param_grid_rf, 
                                                                                          log_transform = False, scale = False)

            # # add the results to the dataframe
            new_row = pd.DataFrame([{
                "n_features": n_features, 
                "n_conv": n_conv, 
                "n_dense": n_dense, 
                "mae": mae
                # "mpe": mpe, 
                # "gdp_change": gdp_per_change
            }])
            
            cnn_results = pd.concat([cnn_results, new_row], ignore_index=True)

            print("Results for n_features = ", n_features, "n_conv = ", n_conv, "n_dense = ", n_dense)

cnn_results



Epoch 1/50


3/3 [==============================] - 4s 757ms/step - loss: 0.0252 - mae: 0.1237 - val_loss: 0.1268 - val_mae: 0.2861
Epoch 2/50
3/3 [==============================] - 1s 507ms/step - loss: 0.0231 - mae: 0.1142 - val_loss: 0.1411 - val_mae: 0.1956
Epoch 3/50
3/3 [==============================] - 1s 497ms/step - loss: 0.0130 - mae: 0.0792 - val_loss: 0.1317 - val_mae: 0.2108
Epoch 4/50
3/3 [==============================] - 1s 504ms/step - loss: 0.0116 - mae: 0.0779 - val_loss: 0.1266 - val_mae: 0.2165
Epoch 5/50
3/3 [==============================] - 1s 510ms/step - loss: 0.0105 - mae: 0.0782 - val_loss: 0.1327 - val_mae: 0.1958
Epoch 6/50
3/3 [==============================] - 2s 534ms/step - loss: 0.0096 - mae: 0.0700 - val_loss: 0.1171 - val_mae: 0.2554
Epoch 7/50
3/3 [==============================] - 1s 476ms/step - loss: 0.0115 - mae: 0.0844 - val_loss: 0.1376 - val_mae: 0.1867
Epoch 8/50
3/3 [==============================] - 1s 480ms/step - loss: 0.0098 - mae: 

In [3]:
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]
ukraine = ukraine[ukraine["year"] < 2023]
ukraine_sum = ukraine[['year', 'region', 'allangle_snow_free_hq_sum', 'allangle_snow_free_hq_mean'] + log_bin_columns]

model = define_cnn(n_features = 4, n_conv = 2, n_dense = 4)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

ukraine_stage_2, ukraine_pred = extract_features_2(model, ukraine, 2022, X, X_pred, 4)
selected_columns =  ["feature_" + str(i) for i in range(1, 5)] + ["allangle_snow_free_hq_sum", "allangle_snow_free_hq_mean"] + log_bin_columns

param_grid_rf = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
        'random_state': [0],  # Ensures reproducibility
}

full_data = pd.concat([ukraine_stage_2, ukraine_pred])
full_data.sort_values(by=['region', 'year'], inplace=True)
full_data = pd.merge(full_data, ukraine_sum, on=['year', 'region'])
full_data_diff = full_data.groupby('region').diff()
full_data_diff['region'] = full_data['region']
full_data_diff['year'] = full_data['year']
full_data_diff.reset_index(drop=True, inplace=True)
full_data_diff = full_data_diff[full_data_diff["year"] != 2012]

train_data_diff = full_data_diff[full_data_diff["year"] != 2022]
pred_data_diff = full_data_diff[full_data_diff["year"] == 2022]
pred_data_diff.reset_index(drop=True, inplace=True)
train_data_diff.reset_index(drop=True, inplace=True)

_, y_pred, best_params = predict_with_model(train_data_diff, pred_data_diff, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False)




Epoch 1/50


4/4 [==============================] - 7s 1s/step - loss: 0.0351 - mae: 0.1525 - val_loss: 0.1645 - val_mae: 0.2200
Epoch 2/50
4/4 [==============================] - 2s 509ms/step - loss: 0.0244 - mae: 0.1199 - val_loss: 0.1634 - val_mae: 0.2186
Epoch 3/50
4/4 [==============================] - 2s 488ms/step - loss: 0.0214 - mae: 0.1085 - val_loss: 0.1485 - val_mae: 0.2285
Epoch 4/50
4/4 [==============================] - 2s 548ms/step - loss: 0.0197 - mae: 0.1036 - val_loss: 0.1483 - val_mae: 0.2034
Epoch 5/50
4/4 [==============================] - 2s 582ms/step - loss: 0.0149 - mae: 0.0889 - val_loss: 0.1389 - val_mae: 0.1959
Epoch 6/50
4/4 [==============================] - 2s 538ms/step - loss: 0.0113 - mae: 0.0738 - val_loss: 0.1335 - val_mae: 0.1920
Epoch 7/50
4/4 [==============================] - 2s 578ms/step - loss: 0.0094 - mae: 0.0664 - val_loss: 0.1220 - val_mae: 0.2182
Epoch 8/50
4/4 [==============================] - 2s 546ms/step - loss: 0.0090 - mae: 0.0

In [4]:
sum(y_pred)/ukraine_2021["real_gdp"].sum()

-0.32132009837230857

In [ ]:
# add the predictions to the dataframe, save
ukraine_2022["gdp_prediction"] = gdp_predictions
ukraine_2022 = ukraine_2022[["region", "year", "gdp_prediction"]]
ukraine_2022.to_csv("data/gdp_predictions_ukraine.csv", index=False)